In [29]:
# ================================================
# 📊 Weekly Sales Report Generator (AI-Enhanced)
# ------------------------------------------------
# This script automates the generation of an insightful, executive-ready
# PDF sales report from uploaded data, tailored for weekly analysis.
#
# 🚀 Key Features:
# - Supports CSV or Excel upload via Google Colab.
# - Cleans and processes key business dimensions.
# - Dynamically filters by Year, Month, Week.
# - Calculates and visualizes:
#     ✓ Sales Executive performance
#     ✓ Customer activity & retention
#     ✓ Cash vs Credit sales + overdue clients
#     ✓ Product category trends & profitability
# - Generates bar charts and embeds into a PDF.
# - AI-generated insights (Gemini API) per section.
# - Auto-downloads a clean, structured PDF report.
#
# 🛠️ Requirements:
# - Google Colab environment (preferred)
# - Internet connection (for Gemini AI)
# - File must include at least the following columns:
#   ["Date", "Week", "Month", "Year", "Customers", "Sales Executive",
#    "Transaction Type", "Amount", "Qty.", "Item Group", "Due Date"]
#
# ✅ Usage:
# 1. Run each cell sequentially.
# 2. Upload your weekly sales file (CSV/Excel).
# 3. Choose Year–Month–Week → Generate report.
# 4. Download auto-generated PDF with embedded insights.
#
# =================================================

In [ ]:
# --- Install dependencies (run once) ---
!pip install -q fpdf matplotlib openpyxl requests ipywidgets

# --- Imports ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fpdf import FPDF
from google.colab import files
from IPython.display import display, clear_output
import ipywidgets as widgets
import unicodedata
from datetime import datetime
import os, json, requests

# --- Gemini API Insight Helper ---
def get_gemini_insight(prompt, api_key):
    url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"
    headers = {"Content-Type": "application/json"}
    data = {"contents": [{"parts": [{"text": prompt}]}]}
    response = requests.post(f"{url}?key={api_key}", headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        return response.json()["candidates"][0]["content"]["parts"][0]["text"]
    else:
        return f"[Error {response.status_code}] {response.text}"

# --- API Key ---
GEMINI_API_KEY = "your_API_Key"  # Replace with your own key

# --- Clean Text for PDF ---
def clean_text(text):
    if pd.isna(text):
        return ""
    return unicodedata.normalize("NFKD", str(text)).encode("ascii", "ignore").decode("ascii")

# --- Upload File ---
print("Upload your sales report file (.csv or .xlsx):")
uploaded = files.upload()
file_path = list(uploaded.keys())[0]

# --- Load Data ---
if file_path.lower().endswith('.csv'):
    df = pd.read_csv(file_path, encoding='latin1', parse_dates=["Date"], dayfirst=True)
else:
    df = pd.read_excel(file_path)
    df.columns = df.columns.str.strip()
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, errors='coerce')

df.columns = df.columns.str.strip()

# --- Format Fixes ---
for col in ["Qty.", "Amount", "Unit Price"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

text_cols = ["Transaction Type", "Transaction Type 1", "Customers", "Item Group", "Sales Executive", "Company", "Vch/Bill No", "Remarks"]
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.upper()

df["Month"] = df["Month"].str.strip().str.title()

# --- Month Map ---
month_to_num = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}

# --- Exclude Credit Notes ---
df_sales = df[~df["Transaction Type"].str.contains("Credit Note", case=False, na=False)]

# --- Get Period Options ---
available_years = sorted(df_sales["Year"].dropna().unique().astype(int).tolist())
available_months = sorted(df_sales["Month"].dropna().unique().tolist(), key=lambda m: month_to_num[m])
available_weeks = sorted(np.unique(df_sales["Week"].dropna().astype(int)).tolist())

# --- User-Friendly Dropdown Selection ---
print("📅 Please select the reporting period:")

year_dropdown = widgets.Dropdown(options=available_years, description='Year:')
month_dropdown = widgets.Dropdown(options=available_months, description='Month:')
week_dropdown = widgets.Dropdown(options=available_weeks, description='Week:')
generate_button = widgets.Button(description="📊 Generate Report", button_style='success')

display(year_dropdown, month_dropdown, week_dropdown, generate_button)

selection = {}

def on_button_click(b):
    selection['year'] = year_dropdown.value
    selection['month'] = month_dropdown.value
    selection['week'] = week_dropdown.value
    clear_output()
    print("✅ Selection received:")
    print(f"  Year:  {selection['year']}")
    print(f"  Month: {selection['month']}")
    print(f"  Week:  {selection['week']}")
    print("\n▶️ Now run the **next cell** to generate the report.")

generate_button.on_click(on_button_click)

✅ Selection received:
  Year:  2025
  Month: June
  Week:  5

▶️ Now run the **next cell** to generate the report.


In [33]:
# --- Ensure selection has been made ---
if not selection:
    raise RuntimeError("Please run the previous cell and click the 'Generate Report' button to make a selection.")

# --- Use selections ---
year_input = selection['year']
month_input = selection['month']
week_input = selection['week']

# --- Filter by Selection ---
filtered_df = df_sales[
    (df_sales["Year"] == year_input) &
    (df_sales["Month"] == month_input) &
    (df_sales["Week"] == week_input)
]

if filtered_df.empty:
    raise ValueError("No data found for the selected Year-Month-Week combination.")

weekly_df = filtered_df.copy()

# --- Labels ---
latest_year = year_input
latest_month_name = month_input
latest_week = week_input

# --- Section 1: Executive ---
cash_txn = weekly_df[weekly_df["Transaction Type 1"] == "CASH"]
cash_exec_counts = cash_txn.groupby("Sales Executive").size()
top_cash_exec = cash_exec_counts.idxmax() if not cash_exec_counts.empty else "N/A"
top_cash_exec_count = cash_exec_counts.max() if not cash_exec_counts.empty else 0

# --- Section 2: Customers ---
cutoff_60 = weekly_df["Date"].max() - pd.Timedelta(days=60)
cutoff_90 = weekly_df["Date"].max() - pd.Timedelta(days=90)
active_60 = set(df_sales[df_sales["Date"] > cutoff_60]["Customers"].unique())
active_90 = set(df_sales[df_sales["Date"] > cutoff_90]["Customers"].unique())
all_customers = set(df_sales["Customers"].unique())
inactive_60 = list(all_customers - active_60)
inactive_90 = list(all_customers - active_90)

top_10_customers = weekly_df.groupby("Customers")["Amount"].sum().nlargest(10)
new_customers = weekly_df["Customers"].unique()

# --- Section 3: Cash vs Credit ---

# Aggregate cash vs credit before plotting
weekly_cc = weekly_df.groupby("Transaction Type 1")["Amount"].sum()

# Change bar colors: Cash = Green, Credit = Orange
weekly_cc.plot(kind="bar", figsize=(6,4), color=["green", "orange"])
plt.title("Cash vs Credit Sales This Week")
plt.ylabel("LKR")
plt.tight_layout()
plt.savefig("/content/weekly_cash_credit.png")
plt.close()

# Format due dates and identify overdue customers
weekly_df.loc[:, "Due Date"] = pd.to_datetime(weekly_df["Due Date"], errors='coerce')
today = pd.Timestamp(datetime.today())
overdue_customers = weekly_df[
    (weekly_df["Due Date"] < today) & (weekly_df["Transaction Type 1"] == "CREDIT")
]["Customers"].unique()

# --- Section 4: Inventory ---
top_items = weekly_df.groupby("Item Group")["Qty."].sum().nlargest(10)
category_profit = weekly_df.groupby("Item Group")["Amount"].sum().sort_values(ascending=False)

top_items.plot(kind='bar', color='skyblue', figsize=(8,4))
plt.title(f"Top Sold Item Groups - Week {latest_week}")
plt.ylabel("Quantity")
plt.tight_layout()
plt.savefig("/content/top_items_week.png")
plt.close()

# --- PDF Generation ---
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# --- Title Page ---
pdf.add_page()
pdf.set_font("Arial", "B", 16)
pdf.cell(0, 10, clean_text(f"Weekly Sales Report - Week {latest_week}, {latest_month_name} {latest_year}"), ln=True, align="C")

pdf.set_font("Arial", "", 12)
pdf.ln(5)
pdf.cell(0, 8, clean_text(f"Total Transactions: {len(weekly_df)}"), ln=True)
pdf.cell(0, 8, clean_text(f"Total Sales: LKR {weekly_df['Amount'].sum():,.2f}"), ln=True)

# === Section 1: Sales Executive Performance ===
pdf.add_page()
pdf.set_font("Arial", "B", 14)
pdf.cell(0, 10, "1. Sales Executive Performance", ln=True)
pdf.set_font("Arial", "", 12)

sec1 = (
    f"- Top Sales Executive: {top_cash_exec} with {top_cash_exec_count} cash transactions\n"
)
pdf.multi_cell(0, 8, clean_text(sec1))

ai_prompt_1 = (
    f"Given the sales executive performance below, provide a concise, actionable summary "
    f"with recommendations limited to 100 words to improve sales and motivate the team.\n\n{sec1}"
)
pdf.multi_cell(0, 8, clean_text("\nAI Insight:\n" + get_gemini_insight(ai_prompt_1, GEMINI_API_KEY)))
pdf.ln(6)

# === Section 2: Customer Insights ===
pdf.add_page()
pdf.set_font("Arial", "B", 14)
pdf.cell(0, 10, "2. Customer Insights", ln=True)
pdf.set_font("Arial", "", 12)

inactive_60_count = len(inactive_60)
inactive_90_count = len(inactive_90)
top_10_cust_lines = "\n".join(
    [f"  - {cust}: LKR {top_10_customers[cust]:,.2f}" for cust in top_10_customers.index]
)
new_cust_count = len(new_customers)

sec2 = (
    f"- Customers inactive >60 days: {inactive_60_count}\n"
    f"- Customers inactive >90 days: {inactive_90_count}\n"
    f"- Top 10 Customers this week:\n{top_10_cust_lines}\n"
    f"- New Customers this Week: {new_cust_count}"
)
pdf.multi_cell(0, 8, clean_text(sec2))

ai_prompt_2 = (
    f"Based on the customer insights below, provide a concise actionable summary "
    f"limited to 100 words with strategic recommendations to increase retention, "
    f"reactivate inactive customers, and grow new customer acquisition.\n\n{sec2}"
)
pdf.multi_cell(0, 8, clean_text("\nAI Insight:\n" + get_gemini_insight(ai_prompt_2, GEMINI_API_KEY)))
pdf.ln(6)

# === Section 3: Cash & Credit Analysis ===
pdf.add_page()
pdf.set_font("Arial", "B", 14)
pdf.cell(0, 10, "3. Cash & Credit Analysis", ln=True)
pdf.set_font("Arial", "", 12)

# Format Overdue Customers as bulleted list
if len(overdue_customers) > 0:
    overdue_list = "\n".join([f"- {cust}" for cust in overdue_customers])
else:
    overdue_list = "None"

weekly_cc_str = "\n".join([f"  - {k}: LKR {v:,.2f}" for k, v in weekly_cc.items()])

sec3 = (
    f"- Overdue Customers:\n{overdue_list}\n"
    f"- Cash vs Credit Sales:\n{weekly_cc_str}"
)
pdf.multi_cell(0, 8, clean_text(sec3))

ai_prompt_3 = (
    f"Review the cash and credit data below and provide a concise, actionable summary "
    f"limited to 100 words with recommendations to improve cash flow, reduce overdue payments, "
    f"and optimize credit risk.\n\n{sec3}"
)
pdf.multi_cell(0, 8, clean_text("\nAI Insight:\n" + get_gemini_insight(ai_prompt_3, GEMINI_API_KEY)))

pdf.image("/content/weekly_cash_credit.png", w=180)
pdf.ln(6)

# === Section 4: Product, Profit & Inventory ===
pdf.add_page()
pdf.set_font("Arial", "B", 14)
pdf.cell(0, 10, "4. Product, Profit & Inventory", ln=True)
pdf.set_font("Arial", "", 12)

top_items_list = "\n".join([f"  - {item}: {int(qty):,}" for item, qty in top_items.items()])
sales_by_cat = "\n".join([f"  - {clean_text(str(k))}: LKR {v:,.2f}" for k, v in category_profit.items()])

sec4 = (
    f"- Top Sold Item Groups:\n{top_items_list}\n"
    f"- Category Profit Breakdown:\n{sales_by_cat}"
)
pdf.multi_cell(0, 8, clean_text(sec4))

ai_prompt_4 = (
    f"Given the product sales and profit data below, provide a concise, actionable summary "
    f"limited to 100 words advising on inventory optimization, product focus, and profit strategies.\n\n{sec4}"
)
pdf.multi_cell(0, 8, clean_text("\nAI Insight:\n" + get_gemini_insight(ai_prompt_4, GEMINI_API_KEY)))

pdf.image("/content/top_items_week.png", w=180)

# --- Save and Download ---
pdf_path = f"/content/Weekly_Sales_Report_Week_{latest_week}_{latest_month_name}_{latest_year}.pdf"
pdf.output(pdf_path)
files.download(pdf_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>